In [1]:
from helper import HelperTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(HelperTest())
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [2]:
from script import ScriptTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(ScriptTest())
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [3]:
from tx import TxTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(TxTest())
unittest.TextTestRunner().run(suite)

...........
----------------------------------------------------------------------
Ran 11 tests in 0.247s

OK


<unittest.runner.TextTestResult run=11 errors=0 failures=0>

In [4]:
# Exercise 1
from binascii import unhexlify
from script import Script

script_pubkey = Script.parse(unhexlify('767695935687'))
print(script_pubkey)

OP_DUP OP_DUP OP_MUL OP_ADD OP_6 OP_EQUAL 


In [5]:
# Exercise 2
from binascii import unhexlify
from script import Script

script_pubkey = Script.parse(unhexlify('6e879169a77ca787'))
print(script_pubkey)

OP_2DUP OP_EQUAL OP_NOT OP_VERIFY OP_SHA1 OP_SWAP OP_SHA1 OP_EQUAL 


In [6]:
from binascii import hexlify, unhexlify
from helper import double_sha256, flip_endian

tx = "0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600"
tx_hash = hexlify(double_sha256(unhexlify(tx)))
print(flip_endian(tx_hash))

452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03


In [7]:
import requests

tx_hash = '452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03'
url = 'https://api.blockcypher.com/v1/btc/main/txs/{}?token=41298c19cc85400da2f1aa620578b096'.format(tx_hash)
tx_json = requests.get(url).json()
print(tx_json)

{'confirmed': '2016-05-05T21:22:12Z', 'hash': '452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03', 'vout_sz': 2, 'lock_time': 410393, 'block_hash': '0000000000000000030bae32d85e56296836ce6036d98abc4962aa966813356e', 'vin_sz': 1, 'confirmations': 77656, 'block_index': 10, 'received': '2016-05-05T21:22:12Z', 'addresses': ['13achaY7hdFTEHCzWC1Cvuo1FDKzDtAvRt', '1GKN6gJBgvet8S92qiQjVxEaVJ5eoJE9s2', '1JAHBxA51vwp5C2zpSB15VbxSZK3hVJs2H'], 'confidence': 1, 'outputs': [{'addresses': ['1JAHBxA51vwp5C2zpSB15VbxSZK3hVJs2H'], 'spent_by': 'c41d126837ce2f6646ae7c46570f55b3352a827e76c9b5b49e0b48e5ab9804f1', 'script': '76a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac', 'script_type': 'pay-to-pubkey-hash', 'value': 32454049}, {'addresses': ['13achaY7hdFTEHCzWC1Cvuo1FDKzDtAvRt'], 'spent_by': '0cad9cb9550a4953f43494ab629de1fa68dbf864ba2e8c0b86d6f0ffd369cf25', 'script': '76a9141c4bc762dd5423e332166702cb75f40df79fea1288ac', 'script_type': 'pay-to-pubkey-hash', 'value': 10011545}], 'inputs'

In [8]:
# Exercise 4

import requests

tx_hash = 'd1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81'
url = 'https://api.blockcypher.com/v1/btc/main/txs/{}?token=41298c19cc85400da2f1aa620578b096'.format(tx_hash)
tx_json = requests.get(url).json()
print(tx_json['outputs'][0]['value'])

42505594


In [9]:
# Exercise 5

from binascii import unhexlify
from io import BytesIO
from tx import Tx

stream = BytesIO(unhexlify('010000000456919960ac691763688d3d3bcea9ad6ecaf875df5339e148a1fc61c6ed7a069e010000006a47304402204585bcdef85e6b1c6af5c2669d4830ff86e42dd205c0e089bc2a821657e951c002201024a10366077f87d6bce1f7100ad8cfa8a064b39d4e8fe4ea13a7b71aa8180f012102f0da57e85eec2934a82a585ea337ce2f4998b50ae699dd79f5880e253dafafb7feffffffeb8f51f4038dc17e6313cf831d4f02281c2a468bde0fafd37f1bf882729e7fd3000000006a47304402207899531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b8461cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba1f686f15f009ded7c62efe85a872e6a19b43c15a2937feffffff567bf40595119d1bb8a3037c356efd56170b64cbcc160fb028fa10704b45d775000000006a47304402204c7c7818424c7f7911da6cddc59655a70af1cb5eaf17c69dadbfc74ffa0b662f02207599e08bc8023693ad4e9527dc42c34210f7a7d1d1ddfc8492b654a11e7620a0012102158b46fbdff65d0172b7989aec8850aa0dae49abfb84c81ae6e5b251a58ace5cfeffffffd63a5e6c16e620f86f375925b21cabaf736c779f88fd04dcad51d26690f7f345010000006a47304402200633ea0d3314bea0d95b3cd8dadb2ef79ea8331ffe1e61f762c0f6daea0fabde022029f23b3e9c30f080446150b23852028751635dcee2be669c2a1686a4b5edf304012103ffd6f4a67e94aba353a00882e563ff2722eb4cff0ad6006e86ee20dfe7520d55feffffff0251430f00000000001976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac005a6202000000001976a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac46430600'))
transaction = Tx.parse(stream)
fee = 0
for tx_in in transaction.tx_ins:
    fee += tx_in.value()
for tx_out in transaction.tx_outs:
    fee -= tx_out.amount
print(fee)

140500


In [10]:
from helper import double_sha256
from binascii import hexlify, unhexlify

modified_tx = '0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000'
h = double_sha256(unhexlify(modified_tx))
z = int.from_bytes(h, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [11]:
from binascii import unhexlify
from ecc import Signature
from io import BytesIO
from tx import Tx

stream = BytesIO(unhexlify('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600'))
transaction = Tx.parse(stream)
der, sighash = transaction.tx_ins[0].der_signature()
signature = Signature.parse(der)
print(signature)

Signature(ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f,7a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed)


In [12]:
from binascii import unhexlify
from ecc import S256Point, Signature
from io import BytesIO
from tx import Tx

stream = BytesIO(unhexlify('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600'))
transaction = Tx.parse(stream)
sec = transaction.tx_ins[0].sec_pubkey()
point = S256Point.parse(sec)
print(point)

Point(49fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a,a56c896489c71dfc65701ce25050f542f336893fb8cd15f4e8e5c124dbf58e47)


In [13]:
# Exercise 7

from binascii import unhexlify
from ecc import S256Point, Signature

sec = unhexlify('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
point = S256Point.parse(sec)

z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
r = 0xed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f
s = 0x7a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed

sig = Signature(r, s)
point.verify(z, sig)

True

In [14]:
# Exercise 8

from binascii import unhexlify
from ecc import S256Point, Signature
from io import BytesIO
from tx import Tx

stream = BytesIO(unhexlify('01000000012f5ab4d2666744a44864a63162060c2ae36ab0a2375b1c2b6b43077ed5dcbed6000000006a473044022034177d53fcb8e8cba62432c5f6cc3d11c16df1db0bce20b874cfc61128b529e1022040c2681a2845f5eb0c46adb89585604f7bf8397b82db3517afb63f8e3d609c990121035e8b10b675477614809f3dde7fd0e33fb898af6d86f51a65a54c838fddd417a5feffffff02c5872e00000000001976a91441b835c78fb1406305727d8925ff315d90f9bbc588acae2e1700000000001976a914c300e84d277c6c7bcf17190ebc4e7744609f8b0c88ac31470600'))
transaction = Tx.parse(stream)
z = transaction.hash_to_sign(0, 1)
der, sighash = transaction.tx_ins[0].der_signature()
signature = Signature.parse(der)
sec = transaction.tx_ins[0].sec_pubkey()
point = S256Point.parse(sec)
point.verify(z, signature)

True